In [1]:
# Load the defaults
import numpy as np
import matplotlib.pyplot as plt
import pylab
import lmfit
plt.style.use("/raven/u/cmor/jupyter/HP_project/my.mplstyle")

In [ ]:
def gene_fit(t, A, g, w, d):  # This is what the diagnostics tool gives you
    return (A - d * t) + ((1 - (A - d * t)) * np.exp(-g * t) * np.cos(w * t))

def edi_fit(t, A, g, w, d, R):  # Edi's fit reported in 
    return A * np.cos(w * t + d) * np.exp(-g * t) + R

def idl_tabulate(x, f, p=5) :  # The function that will help Gao's analysis to integrate and get A
    def newton_cotes(x, f) :
        if x.shape[0] < 2 :
            return 0
        rn = (x.shape[0] - 1) * (x - x[0]) / (x[-1] - x[0])
        weights = scipy.integrate.newton_cotes(rn)[0]
        return (x[-1] - x[0]) / (x.shape[0] - 1) * np.dot(weights, f)
    ret = 0
    for idx in range(0, x.shape[0], p - 1) :
        ret += newton_cotes(x[idx:idx + p], f[idx:idx + p])
    return ret

In [ ]:
def transient_damping(plot='real', log=False):
    
    global Lx, label, title, t0, tf  # Get all this stuff needed
    
    kx = (2 * np.pi / Lx)
    
    # Start plotting details, label and title
#     plt.figure()
#     plt.xlabel('Time units $(a/\\rho_{ti})$')
#     plt.ylabel('Zonal Potential $(\\frac{\langle\phi_{zf}(t)\\rangle}{\langle\phi_{zf}(0)\\rangle})$')
#     plt.axhline(y=0, color='black', linewidth=2, linestyle='dashed')  # Have a reference of zero
#     if title != None:
#         if plot == 'real':  # Normally you choose the real component
#             plt.title('{0} Real component'.format(title))
#         if plot == 'modulus':
#             plt.title('{0} Modulus'.format(title))
    # End plotting details
    
    # Get the data
    gam = pylab.loadtxt(r"timetraceions_act.dat")

#     gam = pylab.loadtxt("{0}/GAM/{1}/gamdiagions_act.dat".format(Lx))  # This, in the case you are in the directory of a NL simulation
#     t_0 = (np.abs(gam_trace[:, 0] - mntime)).argmin()  # These work with the NL data
#     t_f = (np.abs(gam_trace[:, 0] - mxtime)).argmin()

    a = np.divide(gam[:, 1], gam[0, 1], out=np.zeros_like(gam[:, 1]),
                  where=gam[0, 1] != 0)  # The real component
    b = np.divide(gam[:, 2], gam[0, 1], out=np.zeros_like(gam[:, 2]),
                  where=gam[0, 1] != 0)  # Imaginary component
    E = np.sqrt(a**2 + b**2)  # Modulus

    t = gam[:, 0]  # Time

    # Crop the simulation if it's too long. Specify that in the beginning of the function
    if t0 != None:
        ini = (np.abs(t - t0)).argmin()
    else:
        ini = (np.abs(t - t[0])).argmin()
    if tf != None:
        fin = (np.abs(t - tf)).argmin()
    else:
        fin = (np.abs(t - t[-1])).argmin()
        
    mil = (np.abs(t - 500)).argmin()
    res = a[mil]
    
    a = a[ini:fin]
    E = E[ini:fin]
    t = t[ini:fin]
        
#     A_sug, g_sug, w_sug = sugama(kx, filename=r"{0}/gist.dat".format(filename), mode="new")
#     A_gao, g_gao, w_gao = gao(kx, filename=r"{0}/gist.dat".format(filename), mode="new")

    gmodel = lmfit.Model(gene_fit)
    
    pars = gmodel.make_params(A=0.65, g=0.08, w=0.15, d=0.003)
    pars['w'].set(min=0, max=0.7)
    pars['A'].set(min=0, max=2)
    pars['d'].set(min=-np.pi, max=np.pi)
    
#     result = gmodel.fit(a[ini:fin], t=t[ini:fin],
#                         A = A_sug,
#                         g = g_sug,
#                         w = w_sug,
#                         d = 0,
#                         nan_policy='propagate')
    result = gmodel.fit(a, pars, t=t, nan_policy='propagate')

#     print(result.fit_report())
    
    A = result.best_values.get("A")
    g = result.best_values.get("g")
    w = result.best_values.get("w")
    d = result.best_values.get("d")
    
#     if plot == 'real':
#         plt.plot(t, a, label='{0}'.format(label), linewidth=4)
#         plt.plot(t, result.init_fit, label=r"Initial guess")
#         plt.plot(t, result.best_fit, label=r"Best fit")
# #         plt.plot(t, gene_fit(t, A, g, w, d), label="GENE fit")        
# #         plt.plot(t, gene_fit(t, A=A_sug, g=g_sug, w=w_sug, d=0), label="Sugama's")
# #         plt.plot(t, gene_fit(t, A=A_gao, g=g_gao, w=w_gao, d=0), label="Gao's")
#     if plot == 'modulus':
#         plt.plot(t, E, label='{0}'.format(label), color=color)
#     if log == True:
#         plt.yscale('log')

#     plt.legend()
#     plt.show()

    return result.best_values.get("A"), result.best_values.get("g"), result.best_values.get("w"), result.best_values.get("d"), res

In [3]:
def transient_damping_scan(plot='real', log=False, numz=None, A=None, g=None, w=None, d=None):

    global Lx_scan, filename, t0, tf, res  # Get all this stuff needed

    
    # Start plotting details, label and title
    if plot != None:
        plt.figure()
        plt.xlabel('Time units $(a/\\rho_{ti})$')
        plt.ylabel('Zonal Potential $(\\frac{\langle\phi_{zf}(t)\\rangle}{\langle\phi_{zf}(0)\\rangle})$')
        plt.axhline(y=0, color='black', linewidth=2, linestyle='dashed')  # Have a reference of zero
        if plot == 'real':  # Normally you choose the real component
            plt.title('Real component')
        if plot == 'modulus':
            plt.title('Modulus')

    # End plotting details
    all_A = []
    all_g = []
    all_w = []
    all_d = []
    all_r = []
    all_R = []
    
    if numz != None:
        kekke = numz[0]
        keeke = numz[1]
    else:
        kekke = 0
        keeke = len(Lx_scan)

    for n in range(kekke, keeke):

        kx = (2 * np.pi / Lx_scan[n])

        if n <= 8:
            gam = pylab.loadtxt("timetraceions_000{0}.dat".format(n+1))
        elif n >=9 and n <= 98:
            gam = pylab.loadtxt("timetraceions_00{0}.dat".format(n+1))
        else:
            gam = pylab.loadtxt("timetraceions_0{0}.dat".format(n+1))

        a = np.divide(gam[:, 1], gam[0, 1], out=np.zeros_like(gam[:, 1]),
                      where=gam[0, 1] != 0)  # The real component
        b = np.divide(gam[:, 2], gam[0, 1], out=np.zeros_like(gam[:, 2]),
                      where=gam[0, 1] != 0)  # Imaginary component
        E = np.sqrt(a**2 + b**2)  # Modulus

        t = gam[:, 0]  # Time
        
#        a = E

        # Crop the simulation if it's too long. Specify that in the beginning of the function
        if t0 != None:
            ini = (np.abs(t - t0)).argmin()
        else:
            ini = (np.abs(t - t[0])).argmin()
        if tf != None:
            fin = (np.abs(t - tf)).argmin()
        else:
            fin = (np.abs(t - t[-1])).argmin()

#         for kek in range(100, len(np.diff(a))):
#             if np.abs(np.average(np.diff(a[kek-5:kek+5]))) < 0.00001:
#                 break
#         all_r = np.append(all_r, a[kek])
        
        mil = (np.abs(t - res)).argmin()
        all_r = np.append(all_r, a[mil])
        
        a = a[ini:fin]
        E = E[ini:fin]
        t = t[ini:fin]
        
        model = gene_fit

        gmodel = lmfit.Model(model)
        
        if model == gene_fit:
            if n <= 30 :
                pars = gmodel.make_params(A=0.65, g=0.08, w=0.15, d=0.003)
#             elif n > 30 and n < 55:
#                 pars = gmodel.make_params(A=0.60, g=0.09, w=0.11, d=7.7e-04)
#             elif n >= 50 and n < 59:
#                 pars = gmodel.make_params(A=0.4, g=0.12, w=0.09, d=2.45e-04)
#             elif n >= 59 and n < 80:
#                 pars = gmodel.make_params(A=0.29, g=0.18, w=0.06, d=1e-04)
#             elif n >= 82 and n <90:
#                 pars = gmodel.make_params(A=0.12, g=0.27, w=1e-6, d=2.5e-4)
#             elif n >= 90 and n <94:
#                 pars = gmodel.make_params(A=0.14, g=0.26, w=2e-5, d=0.0001)

            if A != None:
                pars = gmodel.make_params(A=A, g=g, w=w, d=d)

            pars['w'].set(min=0, max=0.7)
            pars['A'].set(min=0, max=2)
            pars['d'].set(min=-np.pi, max=np.pi)

            result = gmodel.fit(a, pars, t=t, nan_policy='propagate')

    #         print(result.fit_report())

            all_A = np.append(all_A, result.best_values.get("A"))
            all_g = np.append(all_g, result.best_values.get("g"))
            all_w = np.append(all_w, result.best_values.get("w"))
            all_d = np.append(all_d, result.best_values.get("d"))
            
        elif model == edi_fit:
            pars = gmodel.make_params(A=0.65, g=0.08, w=0.15, d=0.003, R=0.2)

            result = gmodel.fit(a, pars, t=t, nan_policy='propagate')

    #         print(result.fit_report())

            all_A = np.append(all_A, result.best_values.get("A"))
            all_g = np.append(all_g, result.best_values.get("g"))
            all_w = np.append(all_w, result.best_values.get("w"))
            all_d = np.append(all_d, result.best_values.get("d"))
            all_R = np.append(all_R, result.best_values.get("R"))

        if plot == 'real':
            if log == True:
                plt.yscale('log')
#             plt.plot(t, result.init_fit, label=r"Initial guess")
#             plt.plot(t, result.best_fit, linewidth=2, label=r"Best fit")
#             plt.plot(t, gene_fit(t, A=A_sug, g=g_sug, w=w_sug, d=0), label="Sugama's")
#             plt.plot(t, gene_fit(t, A=A_gao, g=g_gao, w=w_gao, d=0), label="Gao's")
            plt.plot(t, a , label='{0}'.format(Lx_scan[n]))  # , color=color)
        if plot == 'modulus':
            if log == True:
                plt.yscale('log')
            plt.plot(t, E, label='{0}'.format(label))  # , color=color))

    if plot != None:
#         plt.legend()
        plt.show()
#     if model == edi_fit:
#         all_r = all_R

    return all_A, all_g, all_w, all_d, all_r

In [ ]:
Lx_scan = [209.440, 104.720, 69.813, 52.360, 41.888, 34.907, 29.920, 26.180, 23.271, 
           20.944, 19.040, 17.453, 16.111, 14.960, 13.963, 13.090, 12.320, 11.636, 11.023, 
           10.472, 9.973, 9.520, 9.106, 8.727, 8.378, 8.055, 7.757, 7.480, 7.222, 6.981, 6.756]

kx = np.zeros_like(Lx_scan)
for n in range(0, len(Lx_scan)):
               kx[n] = 2*np.pi/Lx_scan[n]

In [ ]:
def geometry(filename="../gist_0001"):

    geom = pylab.loadtxt("{0}".format(filename), skiprows=18)
    g11 = geom[:, 0]
    g12 = geom[:, 1]
    g13 = geom[:, 2]
    g22 = geom[:, 3]
    g23 = geom[:, 4]
    g33 = geom[:, 5]
    B = geom[:, 6]
    k_N = geom[:, 7]
    k_G = geom[:, 8]
    J = geom[:, 10]
        
    iota     = 1/data[7]
    B_ref    = data[2]
    B_00     = data[3]
    L = []
    
    ###########################################
    # Calculate the arc length of all the metrics:
    
    phi = z  # This is the toroidal angle
    
    phi_half = list()  # Will store the arc length from phi=0 in the positive direction
    phi_int = list()
    
    # Integrating the Jacobian with respect to theta gives the arc length 
    # as a function of phi
    
    for t in range(len(phi)):
        phi_int.append(simps(J[:t+1], phi[:t+1]))  # This obtains the arc length from the element 1 of the array
    
    for t in range(len(phi[:65])):
        phi_half.append(simps(J[:t+1], phi[:t+1]))  # This obtains it from the outboard midplane (Half)
    
    phi_cent = np.concatenate((np.negative(phi_half[::-1][:-1]),  phi_half[:-1]))  # This mirrors and appends the arc length centered in zero
    dphi = np.diff(phi)
    
    ###########################################

    s = np.diff(g12 / g11) / np.diff(phi_cent)
    s = np.concatenate((s[:64], s[64:65], s[64:]), axis=0)  # Local shear calculation
    
    maxims, minis = peakdet(s, delta=1, x=phi_cent)  # Get the peaks observed in the local shear
    
    color = colourWheel[14]
    R_eff = 3.7282393
    L = abs(minis[0, 0] / 2)
    q_eff = L / (np.pi * R_eff)

    return R_eff, L, q_eff